This code cleans all analytical model outputs and produces all summary statistics and tables reported in the main paper and supplement. It has to be run after the analysis code `OIC-OO v6`, using the same ControlFile and from the same folder; this code will read the output .tab files from the analysis, and will fail if they are not available. See main `Analysis Code` folder for `ReadMe` with explanation of ControlFile format and fields.



In [1]:
import os
import json
import regex
import pandas as pd
import numpy as np

from shutil import copy
from distutils.dir_util import copy_tree


def clean_final(baserunname, scen, name='final', subset=['2003', '2004', '2005']):
    """Clean and process main run output tabfile, yielding tabfile with 
    only variables (no params) and larger timestep; `subset` is used to 
    identify params in the output tabfile, and should *not* include the 
    initial time"""
    table = pd.read_csv(f'{baserunname}_{name}_{scen}.tab', sep='\t', 
                        index_col=0, error_bad_lines=False)

    # Split table along secondary time row
    table1 = table.iloc[:table.index.get_loc('Time'), :]
    table2 = table.iloc[table.index.get_loc('Time'):, :].dropna(axis=1, how='all')
    table2.columns = table2.iloc[0].astype('int').astype('str') # Convert secondary time row to str
    table2 = table2[1:]
    table = pd.concat([table1, table2]) # Concat with joint time str axis to align values by time

    table.dropna(how='all', subset=subset, inplace=True) # Drop constants
    table = table[table.columns[::4]] # Reduce density of time axis to 0.25 years
    table.to_csv(f'{baserunname}_{name}_{scen}_vars.tab', sep='\t')
    
    
def clean_sens(baserunname, scen, fitlist, dropvars=['RepErrRaw']):
    """Clean and process sensitivity output tabfile, yielding cleaned 
    tabfile '_clean' and SimVar-only tabfile '_fits' from `fitlist`, 
    removing `dropvars`"""
    
    # Clean sensitivity data
    senstable = pd.read_csv(f'{baserunname}_sens_{scen}_clean.tab', sep='\t', index_col=[0,1])
    senstable = senstable.reorder_levels(['Var', 'Perc']).sort_index()
    
    # Filter out data for specified variables, by default RepErrRaw
    for var in dropvars:
        filt = pd.Series(~senstable.index.levels[0].str.startswith(var), 
                         index=senstable.index.levels[0])
        senstable = senstable[filt[senstable.index.get_level_values('Var')].values]
    senstable.to_csv(f'{baserunname}_sens_{scen}_clean.tab', sep='\t')
    
    # Extract sensitivity projection fit-to-data
    fits_sens = senstable.loc[[f'SimVar[{var[0]}]' for var in fitlist]]
    fits_sens.to_csv(f'{baserunname}_sens_{scen}_fits.tab', sep='\t')


def insert_sums(tablename, sumlist, sumvars=['SimVar', 'DataVar'], index_col=0):
    """Calculate summed variables and add to tabfile; sums each var in 
    `sumvars` for elements specified in `sumlist`"""
    t = pd.read_csv(tablename, sep='\t', index_col=index_col)
    
    t_dict = {}
    # For each triplet in sumlist, set first elm as sum of other two
    for a, b, c in sumlist:
        for var in sumvars:
            t_dict[f'{var}[{a}]'] = t.loc[f'{var}[{b}]'] + t.loc[f'{var}[{c}]']
    
    # Compile summed variables
    if index_col==0:
        t_sums = pd.concat(t_dict, axis=1).T
    else:
        t_sums = pd.concat(t_dict)
    
    # Merge back in to main dataframe and export to tabfile
    t = pd.concat([t, t_sums]).sort_index()
    t.to_csv(tablename, sep='\t')


def calc_gof(resdf, simvar, datavar):
    """Calculate goodness-of-fit measures for given sim & data vars"""
    # IMPORTANT: cross-screen for missing sim or data values
    sim = resdf.loc[simvar].where(resdf.loc[datavar].notna())
    dat = resdf.loc[datavar].where(resdf.loc[simvar].notna())
    
    # Calculate various GOF stats & return each one
    error = abs(sim - dat)
    maen = error.mean()/dat.mean()
    mape = (error/dat).mean()
    simstd = np.sqrt((sim ** 2).mean() - sim.mean() ** 2)
    datastd = np.sqrt((dat ** 2).mean() - dat.mean() ** 2)
    r2 = (sim.corr(dat)) ** 2
    mse = (error ** 2).mean()
    um = ((sim.mean() - dat.mean()) ** 2/ mse)
    us = ((simstd - datastd) ** 2/ mse)
    uc = (2 * (1 - sim.corr(dat)) * simstd * datastd / mse)
    return maen, mape, r2, mse, um, us, uc
    
    
def get_year_values(table, senstable, var, years, percs, name):
    """Get value and bounds of specified `var` in `years` as text"""
    vartext = [name + '\n'] # Initialise with specified name, varname by default

    # Iterate through years specified and pull values for each
    for year in years:
        val = table.loc[var, year]
        lower = senstable.loc[var, percs[0]].loc[str(float(year))]
        upper = senstable.loc[var, percs[1]].loc[str(float(year))]

        vartext.append(f"{year}\t{val}\t{lower}\t{upper}\n")

    return vartext


def compare_vals(first, second, projvars, projyear, compperc=50.0):
    """Calculate differences between specified `projvars` in `projyear` 
    for `first` and `second`, using `first` as reference values"""
    vals = []
    for file in first, second:
        senstable = pd.read_csv(file, sep='\t', index_col=[0,1])
        senstable = senstable[senstable.columns[::4]]
        senstable.columns = senstable.columns.astype(float).astype(int)
        vals.append([senstable.loc[var, compperc][projyear] for var in projvars])
        del senstable

    # Calculate change in values using first as reference point
    vals_chg = [(var1-var0)/var0 for var0, var1 in zip(vals[0], vals[1])]
    return vals_chg


def compile_sens_panel(baserunname, name, key, scen, outvars, projvars, endyear, 
                       projyear, params=True, dropvars=None):
    """Compile key outcomes panel for sensitivity analysis, specifying 
    run for comparison with `name`, `key` and `scen`, key outcome vars 
    with `outvars` at `endyear` and `projvars` at `projyear`, including 
    parametric sensitivity if `params` is True (and excluding params 
    e.g. from loop knockout with `dropvars`)"""
    
    # Read in base run for comparison and subset key outcome values
    b = pd.read_csv(f'{baserunname}_final_{scen}_vars.tab', sep='\t', index_col=0)
    b = b.loc[outvars + projvars][[endyear, projyear]]

    sensoutdict = {}
    
    # Read in sensitivity run and calculate change in key outcome values vs. base
    t = pd.read_csv(f'{baserunname}_{name}_{key}_{scen}_vars.tab', sep='\t', index_col=0)
    for var in outvars:
        sensoutdict[var] = (t.loc[var, endyear] - b.loc[var, endyear]) / b.loc[var, endyear]
    for var in projvars:
        sensoutdict[var] = (t.loc[var, projyear] - b.loc[var, projyear]) / b.loc[var, projyear]

    # Read in parameter values and calculate sensitivity
    if params:
        paramdf = pd.read_csv(f'{baserunname}_{name}_params.tab', sep='\t', index_col=0)

        pt = paramdf[['Value', key]] # Select values for relevant run
        if dropvars: # Drop specified params (e.g. knocked-out loops)
            pt = pt.drop(dropvars)
        
        pt = pt[pt['Value'] > 0.0001] # Screen out values below 1e-04
        pt['Chg'] = (pt[key] - pt['Value']) / pt['Value']
        sensoutdict['Med elasticity'] = abs(pt['Chg']).median()
        sensoutdict['Max elasticity'] = max(pt['Chg'].min(), pt['Chg'].max(), key=abs)
    else: # Or specify zero elasticity
        sensoutdict['Med elasticity'] = 0
        sensoutdict['Max elasticity'] = 0

    # Subset sensitivity run results
    t = t.loc[:, :endyear]

    # Calculate goodness-of-fit statistics
    t = t[t.columns[::4]] # Subset to each year instead of 0.25 years
    gofs = [[*calc_gof(t, f'SimVar[{elm[0]}]', f'DataVar[{elm[0]}]')] for elm in fitlist]

    gofdf = pd.DataFrame(gofs, index=[elm[0] for elm in fitlist], 
                         columns=['MAEN', 'MAPE', 'R2', 'MSE', 'Um', 'Us', 'Uc'])
    gofdf.loc['Avg'] = gofdf.mean()
    gofdf.to_csv(f'{baserunname}_{name}_{key}_GOF.tab', sep='\t')

    # Pull relevant GOF statistics from GOF stats dataframe
    sensoutdict['Avg MAEN'] = gofdf.loc['Avg', 'MAEN']
    sensoutdict['Max MAEN'] = gofdf['MAEN'].max()
    
    # Return series with each key outcome for the panel
    return pd.Series(sensoutdict)


def strbds_from_perc(perc):
    """Return lower & upper bounds that define `perc` CrI as strings"""
    if perc > 1: # If perc specified as percentage (not decimal)
        return [str(round((0.5 - perc/200), 3)), str(round((0.5 + perc/200), 3))]
    else: # If perc specified as decimal, not 100%
        return [str(round((0.5 - perc/2), 3)), str(round((0.5 + perc/2), 3))]


def get_value(file, varname):
    """General purpose function for reading values from .mdl, .out, etc. 
    files; returns value matching `varname` in a 'var = val' syntax"""
    varregex = regex.compile(r'(?<=([^\w ]|\n)\s?' + regex.escape(varname)
                             + r'\s*=)\s*-?(?:\d*)(\.\d*)?([eE][+\-]?\d+)?')

    with open(file, 'r') as f:
        filetext = f.read()
        value = float((regex.search(varregex, filetext))[0])

    return value

In [2]:
# Read specified controlfile and unpack into variables
controlfilename = input("Enter control file name (with extension):")
cf = json.load(open(controlfilename, 'r'))

for k,v in cf.items():
    exec(k + '=v')

for setting in [analysissettings]:
    for k, v in setting.items():
        exec(k + '=v')

# Initialise base working directory
os.chdir(f"{baserunname}_IterCal")
basedir = os.getcwd()


Enter control file name (with extension):OICC0607.txt


In [3]:
##### DATA FILE PREPARATION AND CLEANING #####

os.chdir(basedir)
os.makedirs('./Results', exist_ok=True)
os.chdir('./Results')

# Copy over all necessary files from other directories
copy(f'../{baserunname}_fits.tab', './')
copy(f'../{baserunname}_params.tab', './')
for cin in (basescens + scenariolist):
    copy(f'../Scenarios/{baserunname}_final_{cin[:-4]}.tab', './')
    copy(f'../Scenarios/{baserunname}_sens_{cin[:-4]}_clean.tab', './')
for cin in basescens[0:2]:
    for proj in proj_subs:
        copy(f'../Scenarios/{baserunname}_final_{cin[:-4]}{proj}.tab', './')
        copy(f'../Scenarios/{baserunname}_sens_{cin[:-4]}{proj}_clean.tab', './')

# Clean & process projection & policy scenario results
for scen in [cin[:-4] for cin in (basescens + scenariolist)]:
    clean_final(baserunname, scen)
    clean_sens(baserunname, scen, fitlist)
    
for scen in [(cin[:-4] + pol[:-4]) for cin in basescens for pol in policylist]:
    clean_final(baserunname, scen)

# Clean & process loop knockout sensitivity results
copy(f'../Sensitivity/{baserunname}_lk_params.tab', './')
for key in lkdict.keys():
    for name in ['lk', 'lk_run']:
        copy(f'../Sensitivity/{baserunname}_{name}_{key}_{basescens[0][:-4]}.tab', './')
        clean_final(baserunname, f'{key}_{basescens[0][:-4]}', name=name)

# Clean & process parametric assumptions sensitivity results
sensdict = dict([[''.join([w[0] for w in regex.findall(r"[\w']+", var)]), var] 
                 for var in sensvars])
copy(f'../Sensitivity/{baserunname}_assm_params.tab', './')
for key in sensdict.keys():
    for sfx in ['_L', '_H']:
        copy(f'../Sensitivity/{baserunname}_assm_{key}{sfx}_{basescens[0][:-4]}.tab', './')
        clean_final(baserunname, f'{key}{sfx}_{basescens[0][:-4]}', name='assm')

In [4]:
##### CALCULATE AGGREGATED VARIABLES, STD ERRS & GOF STATISICS #####

fitdict = dict(fitlist)

# Specify aggregate variables to calculate w/ labels
sumlist = [('ROUT', 'ROUD', 'ROUH'), ('InRT', 'InRM', 'InRD'), ('ODRT', 'ODRB', 'ODSB')]
fitdict['ROUT'] = 'Total Rx opioid use disorder'
fitdict['InRT'] = 'Total Rx misuse initiation'
fitdict['ODRT'] = 'Overdose deaths (Rx + Rx synthetics)'

# Calculate aggregates for various results tabfiles
insert_sums(f'{baserunname}_fits.tab', sumlist)
for cin in basescens:
    insert_sums(f'{baserunname}_final_{cin[:-4]}_vars.tab', sumlist, 
                sumvars=['SimVar', 'DataVar', 'RepVar'])
    insert_sums(f'{baserunname}_sens_{cin[:-4]}_clean.tab', sumlist, 
                sumvars=['SimVar', 'RepVar'], index_col=[0,1])

# Read and append standard error terms where available
tssd = pd.read_excel('../../Time series standard deviations.xlsx', 
                     sheet_name='Summary', index_col=[0,1], header=1)

stderrdict = {} # Initialise container for stderrs
stderrdict['DataErr[InRT]'] = tssd.loc[
    ('Total Rx misuse initiation SAMHSA', 'Standard Error of Weighted Mean')]
stderrdict['DataErr[InHT]'] = tssd.loc[('Total heroin initiation SAMHSA', 'RAND Multiplied SE')]
stderrdict['DataErr[ROUT]'] = (
    tssd.loc[('Rx OUD no PY heroin NSDUH', 'Standard Error of Weighted Mean')] 
    + tssd.loc[('Rx OUD + H NSDUH RAND', 'RAND Multiplied SE')])
stderrdict['DataErr[HUD]'] = tssd.loc[('HUD NSDUH RAND', 'RAND Multiplied SE')]

stderrs = pd.concat(stderrdict, axis=1).T # Concatenate stderr series and transpose
stderrs.columns = stderrs.columns.astype('str')

fits = pd.read_csv(f'{baserunname}_fits.tab', sep='\t', index_col=0)
fits = pd.concat([fits, stderrs])

fits.to_csv(f'{baserunname}_fits.tab', sep='\t')

# Calculate goodness-of-fit statistics
fits = fits[fits.columns[::4]]
gofs = [[*calc_gof(fits, f'SimVar[{elm}]', f'DataVar[{elm}]')] for elm in fitdict.keys()]

gofdf = pd.DataFrame(gofs, index=fitdict.values(), 
                     columns=['MAEN', 'MAPE', 'R2', 'MSE', 'Um', 'Us', 'Uc'])
gofdf.loc['Average'] = gofdf.iloc[0:-3].mean() # Leave out calculated aggregates from average
gofdf.to_csv(f'{baserunname}_GOF.tab', sep='\t')
display(gofdf)

,MAEN,MAPE,R2,MSE,Um,Us,Uc
Rx misuse,0.071125,0.079697,0.890668,5.366574e+11,0.010909,0.497811,0.491280
Rx OUD no heroin,0.086075,0.095445,0.796060,3.752106e+10,0.005035,0.002849,0.992115
Rx OUD with heroin,0.250255,0.321622,0.718206,2.142211e+09,0.001310,0.205208,0.793482
Nondisordered heroin use,0.236346,0.241902,0.330895,2.397396e+10,0.006810,0.109805,0.883385
Heroin use disorder,0.093148,0.102743,0.900277,2.667174e+10,0.006200,0.001413,0.992387
MOUD Tx (buprenorphine),0.029680,0.052923,0.997448,1.820731e+08,0.003098,0.010280,0.986622
MOUD Tx (methadone),0.015241,0.024759,0.953559,2.403119e+08,0.025943,0.094531,0.879527
MOUD Tx (Vivitrol),0.024151,0.042320,0.999733,8.695079e+04,0.283690,0.371786,0.344523
Rx misuse initiation (own Rx),0.088788,0.099504,0.484510,1.173057e+09,0.003198,0.002823,0.993979
Rx misuse initiation (diverted),0.044927,0.048995,0.946574,7.830677e+09,0.000353,0.000080,0.999566


In [5]:
##### COMPILE AND EXPORT INPUT VALUES AND SELECTED YEAR-BY-YEAR VALUES #####

# Assemble input time series projection values
t = pd.read_csv(f'{baserunname}_final_Base_vars.tab', sep='\t', index_col=0)

inputslist = ['Input\t2019\t2031\n'] # Initialise with column labels
for proj in proj_subs:
    val2019 = t.loc[f'Projection output data[{proj}]', '2019']
    val2031 = t.loc[f'Projection output data[{proj}]', '2031']
    inputslist.append(f'{proj}\t{val2019}\t{val2031}\n')
    
with open(f'{baserunname}_inputs.tab', 'w') as f:
    f.writelines(inputslist)
del t

# Compile yearvals output for specified variables and years from sensitivity projections
t = pd.read_csv(f'{baserunname}_final_{basescens[0][:-4]}_vars.tab', sep='\t', index_col=0)
s = pd.read_csv(f'{baserunname}_sens_{basescens[0][:-4]}_clean.tab', sep='\t', index_col=[0,1])

vartext = [f'Year\tVal\t{yv_percs[0]}\t{yv_percs[1]}\n'] # Initialise with column labels
for var in yearvals:
    vartext.extend(get_year_values(t, s, var, years, yv_percs, var))

# Get projection end values
for var in projvars:
    vartext.extend(get_year_values(t, s, var, [str(projyear)], yv_percs, var))
    
# Add prior values
for prior in priorlist:
    vartext.extend(get_year_values(t, s, f'SimPrior[{prior[0]}]', [prior[1]], yv_percs, prior[2]))

with open(f'{baserunname}_yearvals.tab', 'w') as f:
    f.writelines(vartext)

In [6]:
##### ALTERNATIVE PROJECTION ASSUMPTIONS SENSITIVITY ANALYSIS #####

dflist = [] # Initialise empty container

# Iterate through using each basescen as reference point
for cin in basescens[0:2]:
    first = f'{baserunname}_sens_{cin[:-4]}_clean.tab' # Specify reference scenario

    # Calculate comparison for each projection assumption
    vals_chgs = []
    for proj in proj_subs:
        second = f'{baserunname}_sens_{cin[:-4]}{proj}_clean.tab'
        vals_chgs.append(compare_vals(first, second, projvars, projyear))
    dflist.append(pd.DataFrame(vals_chgs, index=proj_subs, columns=projvars))

avgchgdf = (dflist[0] - dflist[1]) / 2 # NOTE: expressed as delta from basescens[0] to [1]
dflist.append(avgchgdf)

# Assemble and export comparison results
cols = [f'{cin} {var}' for cin in ['Base', 'Cnst', 'Avg'] for var in projvars]
chgsdf = pd.concat(dflist, axis=1)
chgsdf.loc['MAC'] = abs(chgsdf).mean() # Calculate mean absolute change
chgsdf.columns = cols
chgsdf.to_csv(f'{baserunname}_proj_changes.tab', sep='\t')

chgsdf

,Base Projected cumulative overdose deaths,Base Projected cumulative UD person years,Cnst Projected cumulative overdose deaths,Cnst Projected cumulative UD person years,Avg Projected cumulative overdose deaths,Avg Projected cumulative UD person years
Fent,-0.140779,0.007408,0.161002,-0.006690,-0.150890,0.007049
HPI,0.000002,0.000000,-0.000002,0.000000,0.000002,0.000000
MME,0.012142,0.033729,-0.018495,-0.045088,0.015318,0.039409
BMDCap,0.000643,0.000077,-0.000910,-0.000220,0.000776,0.000149
MMTCap,0.011912,0.002327,-0.012595,-0.002237,0.012253,0.002282
VivCap,0.002333,0.001540,-0.002422,-0.001276,0.002378,0.001408
NxKD,0.048595,-0.002467,-0.041867,0.001809,0.045231,-0.002138
PtRx,0.026089,0.045623,-0.032628,-0.055078,0.029359,0.050350
ADF,-0.000065,0.000000,0.000011,0.000000,-0.000038,0.000000
RxPP,-0.003076,-0.004584,0.005974,0.009970,-0.004525,-0.007277


In [7]:
##### CLEAN THIS UP MORE / RATIONALISE CONTROLFILE #####
##### PRODUCE SUMMARY TABLES FROM POLICY ANALYSIS #####

annvars = ['Projected total overdose deaths', 'Projected total with UD']
cumvars = ['Projected cumulative overdose deaths', 'Projected cumulative UD person years']
polstart = 2021
quants = [0.025, 0.05, 0.5, 0.9, 0.975]

# Process annual and cumulative main results for each scenario and baseline case
for cin in basescens:
    # Read in baseline results
    b = pd.read_csv(f'{baserunname}_final_{cin[:-4]}_vars.tab', sep='\t', index_col=0)
    resdict = {'Baseline': b.loc[annvars]}
    curdict = {'Baseline': b.loc[cumvars]}
    cumdf = pd.DataFrame(columns=cumvars) # Initialise container dataframe
    cumdf.loc['Baseline'] = [b.loc[var, str(projyear)] - b.loc[var, str(polstart)] 
                             for var in cumvars] # Re-zero to polstart year value
    del b # Clear results to free up memory

    for pol in policylist:
        # Read in results for each scenario
        scen = cin[:-4] + pol[:-4]
        t = pd.read_csv(f'{baserunname}_final_{scen}_vars.tab', sep='\t', index_col=0)
        resdict[pol[:-4]] = t.loc[annvars]
        curdict[pol[:-4]] = t.loc[cumvars]
        cumdf.loc[pol[:-4]] = [t.loc[var, str(projyear)] - t.loc[var, str(polstart)] 
                               for var in cumvars] # Re-zero to polstart year value
        
    # Compile cumulative and annual results dataframes
    resdf = pd.concat(resdict, names=['Scenario', 'Var'])
    curdf = pd.concat(curdict, names=['Scenario', 'Var'])
    curdf = curdf.subtract(curdf[str(polstart)], axis=0)
    resdf = pd.concat([resdf, curdf])
    resdf = resdf.reorder_levels(['Var', 'Scenario']).sort_index()
    resdf = resdf.loc[:, str(polstart):]
    
    # Calculate % changes
    chgdict = {}
    for var in annvars: # Calculate and append for annual results
        chgvar = f'% change in {var}'
        chgdict[chgvar] = ((resdf.loc[var] - resdf.loc[(var, 'Baseline')]) 
                           / resdf.loc[(var, 'Baseline')]) # Calculate % change from baseline
    chgdf = pd.concat(chgdict)
    resdf = resdf.append(chgdf)
    
    for var in cumvars: # Calculate and append for cumulative results
        chgvar = f'% change in {var}'
        cumdf[chgvar] = (cumdf[var] - cumdf.loc['Baseline', var])/ cumdf.loc['Baseline', var]

    # Rename scenarios with specified labels
    resdf.rename(polnames, inplace=True)
    cumdf.rename(polnames, inplace=True)

    resdf.to_csv(f'{baserunname}_{cin[:-4]}_PolRes.tab', sep='\t')
    cumdf.to_csv(f'{baserunname}_{cin[:-4]}_PolTot.tab', sep='\t')

    
# Process annual results with CrI quantiles from full sensitivity sample
pollist = [f'{basescens[0][:-4]}{cin[:-4]}' for cin in policylist] # Compile list of scenarios

poldict = {} # Initialise container for relevant results
# Add main and sens results for each scenario to container
for scen in pollist:
    t = pd.read_csv(f'{baserunname}_final_{scen}_vars.tab', sep='\t', index_col=0)
    s = pd.read_csv(f'{baserunname}_sens_{scen}_clean.tab', sep='\t', index_col=[0,1])
    t.columns = t.columns.astype(float)
    s.columns = s.columns.astype(float)
    
    scendict = {}
    for var in polvars: # Add expected values from baserun to sensitivity dataframe
        scendict[var] = s.loc[var].sort_values(polstart)
        scendict[var].loc['EV'] = t.loc[var]
    poldict[scen] = pd.concat(scendict, keys=polvars, names=['Var', 'Run'])
    del s, t # Clear results to free up memory

# Compile new dataframe of scenario results with full sample
projtable = pd.concat(poldict, names=['Scen', 'Var', 'Run'])
projtable.to_csv(f'{baserunname}_polprojraw.tab', sep='\t')

p = projtable.loc[:, polstart:] # Subset results to relevant years
p.index = p.index.droplevel('Run')

# Calculate annual value quantiles at each time step based on full sample
polprojdict = {}
for scen in pollist[1:]:
    scenpercdict = {}
    for var in polvars:
        scenpercdict[var] = p.loc[(scen, var)].iloc[:-1].quantile(quants)
        scenpercdict[var].loc['EV'] = p.loc[(scen, var)].iloc[-1]
    
    polprojdict[scen] = pd.concat(scenpercdict, keys=polvars, names=['Var', 'Perc'])

# Calculate % change quantiles at each time step based on full sample
polpercdict = {}
for scen in pollist[1:]:
    # First calculate % change across the entire sample
    percs = (p.loc[scen] - p.loc[pollist[0]])/p.loc[pollist[0]]
    
    # Then take quantiles for the % change value at each time step
    scenpercdict = {}
    for var in polvars:
        scenpercdict[var] = percs.loc[var].iloc[:-1].quantile(quants)
        scenpercdict[var].loc['EV'] = percs.loc[var].iloc[-1]
    
    polpercdict[scen] = pd.concat(scenpercdict, keys=polvars, names=['Var', 'Perc'])

# Rename scenarios with specified labels
polrenames = dict([[scen, polnames[cin[:-4]]] for scen, cin in zip(pollist, policylist)])

# Save compiled tables of annual value and % change quantiles
polprojtable = pd.concat(polprojdict, names=['Scen', 'Var', 'Perc'])
polprojtable.rename(index=polrenames, level=0, inplace=True)
polprojtable.to_csv(f'{baserunname}_polprojann.tab', sep='\t')

polperctable = pd.concat(polpercdict, names=['Scen', 'Var', 'Perc'])
polperctable.rename(index=polrenames, level=0, inplace=True)
polperctable.to_csv(f'{baserunname}_polprojperc.tab', sep='\t')

In [8]:
##### LOOP KNOCKOUT ANALYSIS PANEL #####

# Set up labels for loop knockout keys
lknamedict = {'av': 'Availability', 'pr': 'Perceived risk', 'si': 'Social influence'}

lkdfdict = {} # Initiatlise container for results

# Iterate through loop knockout keys and compile deactivated and re-estimated results from each
for key in lkdict.keys():
    lkdfdict['Deactivated ' + lknamedict[key]] = compile_sens_panel(
        baserunname, 'lk_run', key, basescens[0][:-4], outvars, projvars, 
        str(endyear), str(projyear), dropvars=lkdict[key], params=False)
    lkdfdict['Recalibrated w/o ' + lknamedict[key]] = compile_sens_panel(
        baserunname, 'lk', key, basescens[0][:-4], outvars, projvars, 
        str(endyear), str(projyear), dropvars=lkdict[key])

# Compile and export results
lkdf = pd.concat(lkdfdict, axis=1).T
lkdf.to_csv(f'{baserunname}_lk_sens.tab', sep='\t')
lkdf

,Cumulative overdose deaths,Cumulative UD person years,Projected cumulative overdose deaths,Projected cumulative UD person years,Med elasticity,Max elasticity,Avg MAEN,Max MAEN
Deactivated Availability,-0.329865,-0.263021,-0.478979,-0.284717,0.000000,0.000000,0.282527,0.614946
Recalibrated w/o Availability,-0.000462,-0.006230,0.177265,0.241087,0.062402,5.358755,0.136099,0.318448
Deactivated Perceived risk,1.485628,0.629486,4.970749,1.938704,0.000000,0.000000,1.682994,8.292391
Recalibrated w/o Perceived risk,-0.006757,0.004434,0.152970,0.000174,0.142474,24.000000,0.150812,0.272562
Deactivated Social influence,-0.191226,-0.130264,-0.208911,-0.054863,0.000000,0.000000,0.222548,0.511462
Recalibrated w/o Social influence,-0.003482,0.004694,0.234702,0.020750,0.077083,17.657954,0.132263,0.294124


In [9]:
##### PARAMETRIC ASSUMPTIONS SENSITIVITY ANALYSIS PANEL #####

# Compile runnames from variable names in sensvars
sensdict = dict([[''.join([w[0] for w in regex.findall(r"[\w']+", var)]), var] 
                 for var in sensvars])

assmdfdict = {} # Initiatlise container for results

# Iterate through sensvars names and compile results from each
for key in sensdict.keys():
    # Compile high and low scenario results panels
    high = compile_sens_panel(baserunname, 'assm', f'{key}_L', basescens[0][:-4], outvars, 
                              projvars, str(endyear), str(projyear))
    low = compile_sens_panel(baserunname, 'assm', f'{key}_H', basescens[0][:-4], outvars, 
                             projvars, str(endyear), str(projyear))
    
    # Concatenate and take average
    var = pd.concat({'H': high, 'L': low}, axis=1)
    var['avg'] = (abs(var['H']) + abs(var['L'])) / 2 * np.sign(var['H']) # Take sign from H change
    var['avg'].iloc[0:6] = var['avg'].iloc[0:6] / sensrange # Convert to elasticity
    assmdfdict[sensdict[key]] = var['avg']

# Compile and export results
assmdf = pd.concat(assmdfdict, axis=1).T
assmdf.to_csv(f'{baserunname}_assm_sens.tab', sep='\t')
assmdf

,Cumulative overdose deaths,Cumulative UD person years,Projected cumulative overdose deaths,Projected cumulative UD person years,Med elasticity,Max elasticity,Avg MAEN,Max MAEN
ADF substitutability factor,-0.010468,0.003849,-0.067591,-0.059848,0.025532,21.258983,0.122256,0.250147
Average prescription duration,-0.008867,-0.002542,-0.057105,-0.049922,0.021172,23.224836,0.122287,0.250221
Effect of MOUD Tx on OD death rate[Bup],-0.009678,-0.001502,-0.154373,-0.132028,0.124008,36.296228,0.122299,0.249969
Effect of MOUD Tx on OD death rate[MMT],-0.009907,0.002003,-0.100796,-0.082753,0.026165,22.720217,0.122266,0.250043
Effect of MOUD Tx on OD death rate[Viv],-0.008857,0.003622,-0.096558,-0.073532,0.024427,22.171642,0.122275,0.250146
OxyContin withdrawal magnitude,-0.010510,0.005288,-0.178971,-0.121825,0.078171,27.741179,0.122198,0.249778
Perceived risk decrease time,-0.010520,0.002558,-0.120021,-0.095808,0.029782,25.545743,0.122239,0.249946
Perceived risk increase time,-0.013452,0.003497,-0.331149,-0.286739,0.124372,37.923019,0.122323,0.250247
Perceived risk weight NFOD,-0.011840,0.004122,-0.137268,-0.093308,0.018917,14.652762,0.122355,0.250570
Probability OD witnessed,-0.009252,0.001478,-0.106422,-0.149434,0.022688,19.482782,0.122319,0.250082


In [10]:
##### SYNDATA CrI PROCESSING #####

# Read in data
syndf = pd.read_csv(f'{baserunname}_syndata_results.tab', sep='\t', index_col=[0, 1])
syndf = syndf.reorder_levels(['Perc', 'Run']).sort_index()

spdfdict = {} # Initialise container for results

# Create Boolean df tracking which values are within which percent CrIs
for perc in syn_reppercs:
    bds = strbds_from_perc(perc) # Calculate CrI bounds for each percent CrI
    spdfdict[perc] = ((syndf.loc['True'] > syndf.loc[bds[0]]) 
                      & (syndf.loc['True'] < syndf.loc[bds[1]]))

# Calculate distance of estimate from median relative to main CrI
bdsmain = strbds_from_perc(95)
spdfdict[f'dist{syn_mainperc}'] = abs((syndf.loc['Value'] - syndf.loc['True']) / 
                                      (syndf.loc[bdsmain[1]] - syndf.loc[bdsmain[0]]))

# Compile and export percent CrI calculations
synpercdf = pd.concat(spdfdict, names=['Perc', 'Run'])
synpercdf.to_csv(f'{baserunname}_syndata_intervals.tab', sep='\t')

# Collapse Boolean df to get mean percentages within each CrI
means = synpercdf.mean(axis=1).groupby('Perc').mean()
means[f'Dist{syn_mainperc}Med'] = np.nanmedian(spdfdict[f'dist{syn_mainperc}'])
means.to_csv(f'{baserunname}_syndata_means.tab', sep='\t')
means

Perc
50           0.352055
80           0.609589
90           0.710959
95           0.755479
98           0.798630
dist95       0.453847
Dist95Med    0.296072
dtype: float64

In [11]:
##### CALCULATE VALUES FOR SUMMARYTEXT #####

# Pull values for fentanyl counterfactual ODs
t = pd.read_csv(f'{baserunname}_final_{basescens[0][:-4]}_vars.tab', sep='\t', index_col=0)
nft = pd.read_csv(f'{baserunname}_final_{scenariolist[0][:-4]}_vars.tab', sep='\t', index_col=0)
nofentods = nft.loc['Cumulative overdose deaths', '2019']
nofentodsdata = np.sum((t.loc['Total overdose deaths NVSS'] 
                        - t.loc['Total overdose deaths base Rx NVSS'] 
                        - t.loc['Total overdose deaths base heroin NVSS'])[::4])
del t, nft

# Calculate MCMC sample size
mcsample = mcsettings['MCLIMIT'] - mcsettings['MCBURNIN']

# Calculate PSRF percentages below 1.1 and 1.2 key thresholds
mcout = pd.read_csv(f'{baserunname}_main_MC_MCMC_stats.tab', sep='\t', index_col=0)
psrfs = [i for i in mcout.index if 'PSRF' in i]
psrfs.remove('PSRF Payoff')
mcout = mcout.loc[psrfs]
mcout.columns = mcout.columns.astype('float').astype('int')
mcout = mcout[mcout.columns[mcout.columns > mcsettings['MCBURNIN']]]
psrf12 = np.nanmean(mcout < 1.2)
psrf11 = np.nanmean(mcout < 1.1)
del mcout

# Get parameter numbers
t = pd.read_csv(f'{baserunname}_params.tab', sep='\t', index_col=0)
iscs = len([idx for idx in t.index if 'Initial stock correction' in idx])
estpars = len(t.index) - iscs
del t

# Compile summarytext
summarytext = [
    f"Exogenous inputs\t{len(proj_subs)}\n", 
    f"Calibration time series\t{len(fitlist) - 3}\n", 
    f"MCMC total\t{mcsettings['MCLIMIT']}\n", 
    f"MCMC burnin\t{mcsettings['MCBURNIN']}\n", 
    f"MCMC sample\t{mcsample}\n", 
    f"MCMC PSRF < 1.2\t{psrf12}\t< 1.1\t{psrf11}\n", 
    f"Sensitivity sample\t{int(mcsample * samplefrac)}\n", 
    f"Sensitivity analysis range\t{sensrange}\n", 
    f"Syndata sets\t{synsample}\n", 
    f"Estimated parameters (no ISCs)\t{estpars}\n", 
    f"Initial stock corrections\t{iscs}\n", 
    f"Cumulative OD deaths without fentanyl\t{int(nofentods)}\n", 
    f"Cumulative synth-involved OD deaths DATA\t{int(nofentodsdata)}\n", 
    f"Confidence interval estimated params\t{round(param_percs[-1] - param_percs[0], 3)}\n", 
    f"Confidence interval estimated params\t{syn_mainperc/100}\n"
]

# Calculate projection differences for key outcomes expressed as delta from 'base' to 'cnst'
first = f'{baserunname}_sens_{basescens[0][:-4]}_clean.tab'
second = f'{baserunname}_sens_{basescens[1][:-4]}_clean.tab'
basecomps = compare_vals(first, second, projvars, projyear)

summarytext.extend([f"Base-Cnst delta for {var}\t{val}\n" for var, val in zip(projvars, basecomps)])

# Read fixed parameter values from .mdl file
mdl = f"../{simsettings['model']}"
summarytext.append("\n\nFixed parameter values\n")
summarytext.extend([f'{var}\t{get_value(mdl, var)}\n' for var in paramvals])

# Create relative Tx-seeking rate table
ot = 1
ob = get_value(mdl, "Tx seeking fraction Bup Rx OUD")
om = round((ot - ob) * get_value(mdl, "Tx seeking fraction MMT Rx OUD relative"), 5)
ov = round(ot - ob - om, 5)
ht = get_value(mdl, "Tx seeking rate HUD relative to Rx OUD no H")
hb = round(ht * get_value(mdl, "Tx seeking fraction Bup HUD"), 5)
hm = round((ht - hb) * get_value(mdl, "Tx seeking fraction MMT HUD relative"), 5)
hv = round(ht - hb - hm, 5)

summarytext.extend(["\n\n", "Relative Tx seeking rates\n", 
                    f"OUD\t{ot}\t{ob}\t{om}\t{ov}\n", f"HUD\t{ht}\t{hb}\t{hm}\t{hv}\n"])

# Export summary text file
with open(f"{baserunname}_summary.txt", 'w') as summaryfile:
    summaryfile.writelines(summarytext)

display(summarytext)


['Exogenous inputs\t10\n',
 'Calibration time series\t15\n',
 'MCMC total\t2500000\n',
 'MCMC burnin\t1500000\n',
 'MCMC sample\t1000000\n',
 'MCMC PSRF < 1.2\t0.9813200498132005\t< 1.1\t0.9302615193026152\n',
 'Sensitivity sample\t5000\n',
 'Sensitivity analysis range\t0.1\n',
 'Syndata sets\t20\n',
 'Estimated parameters (no ISCs)\t53\n',
 'Initial stock corrections\t20\n',
 'Cumulative OD deaths without fentanyl\t379518\n',
 'Cumulative synth-involved OD deaths DATA\t170563\n',
 'Confidence interval estimated params\t0.9\n',
 'Confidence interval estimated params\t0.95\n',
 'Base-Cnst delta for Projected cumulative overdose deaths\t-0.04764694395625854\n',
 'Base-Cnst delta for Projected cumulative UD person years\t0.09676042242579905\n',
 '\n\nFixed parameter values\n',
 'Perceived risk weight NFOD\t0.1\n',
 'Average prescription duration\t0.059\n',
 'OxyContin withdrawal magnitude\t0.45\n',
 'Tx seeking rate HUD relative to Rx OUD no H\t4.843\n',
 'Bup effective capacity decay con

In [14]:
##### SEND MAIN OUTPUTS TO SUBFOLDER FOR EASY ACCESS #####

os.chdir(basedir)
os.chdir('./Results')
os.makedirs('./ResMain', exist_ok=True)

resmain = ['assm_sens.tab', 'GOF.tab', 'inputs.tab', 'lk_sens.tab', 'params.tab', 
           'polprojann.tab', 'polprojperc.tab', 'proj_changes.tab', 'syndata_means.tab', 
           'yearvals.tab', 'summary.txt']

for res in resmain:
    copy(f'./{baserunname}_{res}', './ResMain')

In [ ]:
000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000